# CDAT Migration Regression Testing Notebook (`.nc` files)

This notebook is used to perform regression testing between the development and
production versions of a diagnostic set.

## How it works

It compares the relative differences (%) between ref and test variables between
the dev and `main` branches.

## How to use

PREREQUISITE: The diagnostic set's netCDF stored in `.json` files in two directories
(dev and `main` branches).

1. Make a copy of this notebook under `auxiliary_tools/cdat_regression_testing/<DIR_NAME>`.
2. Run `mamba create -n cdat_regression_test -y -c conda-forge "python<3.12" xarray netcdf4 dask pandas matplotlib-base ipykernel`
3. Run `mamba activate cdat_regression_test`
4. Update `SET_DIR` and `SET_NAME` in the copy of your notebook.
5. Run all cells IN ORDER.
6. Review results for any outstanding differences (>=1e-5 relative tolerance).
   - Debug these differences (e.g., bug in metrics functions, incorrect variable references, etc.)


## Setup Code


In [22]:
import glob
from typing import List

import numpy as np
import xarray as xr

from e3sm_diags.derivations.derivations import DERIVED_VARIABLES

DEV_DIR = "25-01-10-branch-907-v2-data"
DEV_PATH = f"/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/{DEV_DIR}/"

DEV_GLOB = sorted(glob.glob(DEV_PATH + "**/**/*.nc"))
DEV_NUM_FILES = len(DEV_GLOB)

MAIN_DIR = "main"
MAIN_PATH = f"/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/{MAIN_DIR}/"
MAIN_GLOB = sorted(glob.glob(MAIN_PATH + "**/**/*.nc"))
MAIN_NUM_FILES = len(MAIN_GLOB)


def _remove_unwanted_files(file_glob: List[str]) -> List[str]:
    """Remove files that we don't want to compare.

    * area_mean_time_series -- `main` does not generate netCDF
    * enso_diags -- `main` does not generate netCDF
    * qbo -- variable name differs
    * diurnal_cycle -- variable name differs
    * diff -- comparing the difference between regridded files is not helpful
      between branches because of the influence in floating point errors.
    * ERA5_ext-U10-ANN-global_ref and ERA5_ext-U10-JJA-global_ref -- dev
      branch does not generate these files because it is a model-only run.

    Parameters
    ----------
    file_glob : List[str]
        _description_

    Returns
    -------
    List[str]
        _description_
    """

    new_glob = []

    for fp in file_glob:
        if (
            "area_mean_time_series" in fp
            or "enso_diags" in fp
            or "qbo" in fp
            or "diurnal_cycle" in fp
            or "diff" in fp
            or "ERA5_ext-U10-ANN-global_ref" in fp
            or "ERA5_ext-U10-JJA-global_ref" in fp
        ):
            continue

        new_glob.append(fp)

    return new_glob


DEV_GLOB = _remove_unwanted_files(DEV_GLOB)
MAIN_GLOB = _remove_unwanted_files(MAIN_GLOB)

In [23]:
def _get_relative_diffs():
    # The absolute and relative tolerances for the tests.
    ATOL = 0
    RTOL = 1e-4

    results = {
        "missing_files": [],
        "missing_vars": [],
        "matching_files": [],
        "mismatch_errors": [],
        "not_equal_errors": [],
        "key_errors": [],
    }

    for fp_main in MAIN_GLOB:
        fp_dev = fp_main.replace(MAIN_DIR, DEV_DIR)

        if "annual_cycle_zonal_mean" in fp_main:
            if "test.nc" in fp_main:
                fp_dev = fp_dev.replace("test.nc", "ref.nc")
            elif "ref.nc" in fp_main:
                fp_dev = fp_dev.replace("ref.nc", "test.nc")

        try:
            ds1 = xr.open_dataset(fp_dev)
            ds2 = xr.open_dataset(fp_main)
        except FileNotFoundError as e:
            print(f"    {e}")

            if isinstance(e, FileNotFoundError) or isinstance(e, OSError):
                results["missing_files"].append(fp_dev)

            continue

        var_key = fp_main.split("-")[-3]

        # for 3d vars such as T-200
        var_key.isdigit()
        if var_key.isdigit():
            var_key = fp_main.split("-")[-4]

        dev_data = _get_var_data(ds1, var_key)
        main_data = _get_var_data(ds2, var_key)

        if dev_data is None or main_data is None:
            if dev_data is None:
                results["missing_vars"].append(fp_dev)
            elif main_data is None:
                results["missing_vars"].append(fp_main)

            print("    * Could not find variable key in the dataset(s)")

            continue

        try:
            np.testing.assert_allclose(
                dev_data,
                main_data,
                atol=ATOL,
                rtol=RTOL,
            )
            results["matching_files"].append(fp_main)
        except (KeyError, AssertionError) as e:
            print("Comparing:")
            print(f"    * {fp_dev}")
            print(f"    * {fp_main}")
            print(f"    * var_key: {var_key}")
            msg = str(e)

            print(f"    {msg}")

            if "mismatch" in msg:
                results["mismatch_errors"].append(fp_dev)
            elif "Not equal to tolerance" in msg:
                results["not_equal_errors"].append(fp_dev)

    return results


def _get_var_data(ds: xr.Dataset, var_key: str) -> np.ndarray:
    """Get the variable data using a list of matching keys.

    The `main` branch saves the dataset using the original variable name,
    while the dev branch saves the variable with the derived variable name.
    The dev branch is performing the expected behavior here.

    Parameters
    ----------
    ds : xr.Dataset
        _description_
    var_key : str
        _description_

    Returns
    -------
    np.ndarray
        _description_
    """

    data = None

    try:
        data = ds[var_key].values
    except KeyError:
        try:
            var_keys = DERIVED_VARIABLES[var_key].keys()
        except KeyError:
            var_keys = DERIVED_VARIABLES[var_key.upper()].keys()

        var_keys = [var_key] + list(sum(var_keys, ()))

        for key in var_keys:
            if key in ds.data_vars.keys():
                data = ds[key].values
                break

    return data

In [24]:
def _check_if_files_found():
    if DEV_NUM_FILES == 0 or MAIN_NUM_FILES == 0:
        raise IOError(
            "No files found at DEV_PATH and/or MAIN_PATH. "
            f"Please check {DEV_PATH} and {MAIN_PATH}."
        )


def _check_if_matching_filecount():
    if DEV_NUM_FILES != MAIN_NUM_FILES:
        raise IOError(
            "Number of files do not match at DEV_PATH and MAIN_PATH "
            f"({DEV_NUM_FILES} vs. {MAIN_NUM_FILES})."
        )

    print(f"Matching file count ({DEV_NUM_FILES} and {MAIN_NUM_FILES}).")


def _check_if_missing_files():
    missing_dev_files = []
    missing_main_files = []

    for fp_main in MAIN_GLOB:
        fp_dev = fp_main.replace(MAIN_DIR, DEV_DIR)

        if fp_dev not in DEV_GLOB:
            missing_dev_files.append(fp_dev)

    for fp_dev in DEV_GLOB:
        fp_main = fp_dev.replace(DEV_DIR, MAIN_DIR)

        if fp_main not in MAIN_GLOB:
            missing_main_files.append(fp_main)

    return missing_dev_files, missing_main_files

## 1. Check for matching and equal number of files


In [25]:
_check_if_files_found()

In [26]:
DEV_GLOB = [fp for fp in DEV_GLOB if "diff.nc" not in fp]
MAIN_GLOB = [fp for fp in MAIN_GLOB if "diff.nc" not in fp]

In [27]:
len(DEV_GLOB), len(MAIN_GLOB)

(1254, 1248)

In [28]:
missing_dev_files, missing_main_files = _check_if_missing_files()

print(f"Missing dev files: {len(missing_dev_files)}")
print(f"Missing main files: {len(missing_main_files)}")

Missing dev files: 2
Missing main files: 8


### Check missing main files (not concerned)

Results:

- The missing files are due to a recent .cfg update in [PR #830](https://github.com/E3SM-Project/e3sm_diags/pull/830)
- OMI-MLS-TCO-ANN-60S60N is missing due to recent merge of [PR [#897](https://github.com/E3SM-Project/e3sm_diags/issues/897)]


In [29]:
missing_main_files

['/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/annual_cycle_zonal_mean/MACv2/MACv2-AODVIS-ANNUALCYCLE-global_ref.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/annual_cycle_zonal_mean/MACv2/MACv2-AODVIS-ANNUALCYCLE-global_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/annual_cycle_zonal_mean/MERRA2_Aerosols/MERRA2_Aerosols-AODVIS-ANNUALCYCLE-global_ref.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/annual_cycle_zonal_mean/MERRA2_Aerosols/MERRA2_Aerosols-AODVIS-ANNUALCYCLE-global_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/lat_lon/OMI-MLS/OMI-MLS-TCO-ANN-60S60N_ref.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/lat_lon/OMI-MLS/OMI-MLS-TCO-ANN-60S60N_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/lat_lon/OMI-MLS/OMI-MLS-TCO-JJA-60S60N_ref.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/lat_lon/OMI-MLS/OMI-MLS-TCO-JJA-60S60N_test.nc']

### Check missing dev files:

Results:

- The missing reference files are due to not saving them out to netCDF since they are the same as the test files (skipped, model-only run)


In [30]:
missing_dev_files

['/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/25-01-10-branch-907-v2-data/annual_cycle_zonal_mean/AOD_550/AOD_550-AODVIS-ANNUALCYCLE-global_ref.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/25-01-10-branch-907-v2-data/annual_cycle_zonal_mean/AOD_550/AOD_550-AODVIS-ANNUALCYCLE-global_test.nc']

## 2 Compare the netCDF files between branches

- Compare "ref" and "test" files
- "diff" files are ignored because getting relative diffs for these does not make sense (relative diff will be above tolerance)


In [31]:
results = _get_relative_diffs()

    [Errno 2] No such file or directory: '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/25-01-10-branch-907-v2-data/annual_cycle_zonal_mean/AOD_550/AOD_550-AODVIS-ANNUALCYCLE-global_test.nc'
    [Errno 2] No such file or directory: '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/25-01-10-branch-907-v2-data/annual_cycle_zonal_mean/AOD_550/AOD_550-AODVIS-ANNUALCYCLE-global_ref.nc'
Comparing:
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/25-01-10-branch-907-v2-data/annual_cycle_zonal_mean/CERES-EBAF-TOA-v4.1/ceres_ebaf_toa_v4.1-ALBEDO-ANNUALCYCLE-global_ref.nc
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/annual_cycle_zonal_mean/CERES-EBAF-TOA-v4.1/ceres_ebaf_toa_v4.1-ALBEDO-ANNUALCYCLE-global_test.nc
    * var_key: ALBEDO
    
Not equal to tolerance rtol=0.0001, atol=0

nan location mismatch:
 ACTUAL: array([[0.69877 , 0.695266, 0.68627 , ...,      inf,      inf,      inf],
       [0.712032, 0.706896, 0.69354 , ...,      inf,      inf,      inf],
       [0.765447, 0.7

In [32]:
import pandas as pd

# Statistics
(
    missing_files,
    missing_vars,
    matching_files,
    mismatch_errors,
    not_equal_errors,
    key_errors,
) = results.values()

In [33]:
# Assuming these variables are defined in your notebook
matching_files_count = len(matching_files)
missing_vars_count = len(missing_vars)
mismatch_errors_count = len(mismatch_errors)
not_equal_errors_count = len(not_equal_errors)
key_errors_count = len(key_errors)
missing_files_count = len(missing_files)

sum_files_compared = (
    matching_files_count
    + missing_vars_count
    + mismatch_errors_count
    + not_equal_errors_count
    + key_errors_count
    + missing_files_count
)

pct_match = (matching_files_count / sum_files_compared) * 100

# Collect statistics into a dictionary
statistics = {
    "stat_name": [
        "matching_files_count",
        "missing_vars_count",
        "mismatch_errors_count",
        "not_equal_errors_count",
        "key_errors_count",
        "missing_files_count",
    ],
    "value": [
        matching_files_count,
        missing_vars_count,
        mismatch_errors_count,
        not_equal_errors_count,
        key_errors_count,
        missing_files_count,
    ],
    "pct": [
        matching_files_count / sum_files_compared,
        missing_vars_count / sum_files_compared,
        mismatch_errors_count / sum_files_compared,
        not_equal_errors_count / sum_files_compared,
        key_errors_count / sum_files_compared,
        missing_files_count / sum_files_compared,
    ],
}

# Convert the dictionary to a pandas DataFrame
df = pd.DataFrame(statistics)

# Display the DataFrame
print(df)

                stat_name  value       pct
0    matching_files_count   1093  0.875801
1      missing_vars_count      0  0.000000
2   mismatch_errors_count     10  0.008013
3  not_equal_errors_count    143  0.114583
4        key_errors_count      0  0.000000
5     missing_files_count      2  0.001603


### Missing Files


In [34]:
missing_files

['/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/25-01-10-branch-907-v2-data/annual_cycle_zonal_mean/AOD_550/AOD_550-AODVIS-ANNUALCYCLE-global_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/25-01-10-branch-907-v2-data/annual_cycle_zonal_mean/AOD_550/AOD_550-AODVIS-ANNUALCYCLE-global_ref.nc']

- AOD_550 files were retired (https://github.com/E3SM-Project/e3sm_diags/issues/852)


In [35]:
missing_files = []

### `NaN` Mismatching Errors

I found these `nan` mismatch errors occur due to either:

1. Regional subsetting on "ccb" flag in CDAT adding a coordinate points -- removing these coordinates results in matching results
2. Slightly different masking in the data between xCDAT and CDAT via xESMF/ESMF -- same number of nans just slightly shifted over some coordinates points

- Refer to PR [#794](https://github.com/E3SM-Project/e3sm_diags/pull/794)


In [36]:
mismatch_errors

['/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/25-01-10-branch-907-v2-data/annual_cycle_zonal_mean/CERES-EBAF-TOA-v4.1/ceres_ebaf_toa_v4.1-ALBEDO-ANNUALCYCLE-global_ref.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/25-01-10-branch-907-v2-data/lat_lon/ERA5/ERA5-TAUXY-ANN-ocean_ref.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/25-01-10-branch-907-v2-data/lat_lon/ERA5/ERA5-TAUXY-JJA-ocean_ref.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/25-01-10-branch-907-v2-data/lat_lon/ERA5/ERA5-TREFHT-ANN-land_ref.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/25-01-10-branch-907-v2-data/lat_lon/ERA5/ERA5-TREFHT-JJA-land_ref.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/25-01-10-branch-907-v2-data/lat_lon/MERRA2/MERRA2-TAUXY-ANN-ocean_ref.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/25-01-10-branch-907-v2-data/lat_lon/MERRA2/MERRA2-TAUXY-JJA-ocean_ref.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/25-01-10-branch-907-v2-data/lat_lon/MERRA2/MERR

In [37]:
mismatch_errors = [
    f
    for f in mismatch_errors
    # https://github.com/E3SM-Project/e3sm_diags/pull/794
    if "TAUXY" not in f and "ERA5-TREFHT" not in f and "MERRA2-TREFHT" not in f
    # https://github.com/E3SM-Project/e3sm_diags/pull/798#issuecomment-2251287986
    and "ceres_ebaf_toa_v4.1-ALBEDO" not in f
]

In [38]:
mismatch_errors

[]

### Not Equal Errors

- Note, some files are omitted due to known root causes to the diffs (not a concern)


In [39]:
import xarray as xr


def get_stats_for_not_equal_files(filepaths):
    for fp_dev in filepaths:
        fp_main = fp_dev.replace(DEV_DIR, MAIN_DIR)

        if "annual_cycle_zonal_mean" in fp_main:
            if "test.nc" in fp_main:
                fp_dev = fp_dev.replace("test.nc", "ref.nc")
            elif "ref.nc" in fp_main:
                fp_dev = fp_dev.replace("ref.nc", "test.nc")

        ds1 = xr.open_dataset(fp_dev)
        ds2 = xr.open_dataset(fp_main)

        var_key = fp_main.split("-")[-3]

        # for 3d vars such as T-200
        var_key.isdigit()
        if var_key.isdigit():
            var_key = fp_main.split("-")[-4]

        dev_mean = ds1[var_key].mean().item()
        main_mean = ds2[var_key].mean().item()

        dev_sum = ds1[var_key].sum().item()
        main_sum = ds2[var_key].sum().item()

        print(f"Checking variable {var_key}")
        print(f"Dev Path: {fp_dev}")
        print(f"Main Path: {fp_main}")
        print("-------------------------------------")

        mean_diff = dev_mean - main_mean
        sum_diff = dev_sum - main_sum

        absolute_mean_diff = abs(mean_diff)
        absolute_sum_diff = abs(sum_diff)

        relative_mean_diff = (
            absolute_mean_diff / abs(main_mean) if main_mean != 0 else float("inf")
        )
        relative_sum_diff = (
            absolute_sum_diff / abs(main_sum) if main_sum != 0 else float("inf")
        )
        dev_min = ds1[var_key].min().item()
        dev_max = ds1[var_key].max().item()

        main_min = ds2[var_key].min().item()
        main_max = ds2[var_key].max().item()

        print(f"* Min - dev: {dev_min:.6f}, main: {main_min:.6f}")
        print(f"* Max - dev: {dev_max:.6f}, main: {main_max:.6f}")

        print(f"* Mean - dev: {dev_mean:.6f}, main: {main_mean:.6f}")
        print(f"    * Absolute Mean Diff: {absolute_mean_diff}")
        print(f"    * Relative Mean Diff: {relative_mean_diff * 100:.6f}%")

        print(f"* Sum - dev: {dev_sum:.6f}, main: {main_sum:.6f}")
        print(f"    * Absolute Sum Diff: {absolute_sum_diff}")
        print(f"    * Relative Sum Diff: {relative_sum_diff * 100:.6f}%")

In [40]:
get_stats_for_not_equal_files(not_equal_errors)

Checking variable TREFHT
Dev Path: /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/25-01-10-branch-907-v2-data/annual_cycle_zonal_mean/ERA5/ERA5-TREFHT-ANNUALCYCLE-global_ref.nc
Main Path: /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/annual_cycle_zonal_mean/ERA5/ERA5-TREFHT-ANNUALCYCLE-global_test.nc
-------------------------------------
* Min - dev: -54.642141, main: -54.642134
* Max - dev: 27.755421, main: 27.755427
* Mean - dev: 5.311142, main: 5.311148
    * Absolute Mean Diff: 5.963206636039331e-06
    * Relative Mean Diff: 0.000112%
* Sum - dev: 45951.999787, main: 45952.051381
    * Absolute Sum Diff: 0.051593663563835435
    * Relative Sum Diff: 0.000112%
Checking variable TREFHT
Dev Path: /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/25-01-10-branch-907-v2-data/annual_cycle_zonal_mean/ERA5/ERA5-TREFHT-ANNUALCYCLE-global_test.nc
Main Path: /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/annual_cycle_zonal_mean/ERA5/ERA5-TREFHT-ANNUALCYCLE-global_ref.nc
--------------

## Let's remove all of the files that have close stats and minimal differences.


In [41]:
not_equal_errors = [
    f
    for f in not_equal_errors
    # https://github.com/E3SM-Project/e3sm_diags/issues/797
    if "MISRCOSP-CLDLOW_TAU1.3_9.4_MISR" not in f
    and "MISRCOSP-CLDLOW_TAU1.3_MISR" not in f
    and "MISRCOSP-CLDLOW_TAU9.4_MISR" not in f
    and "MISRCOSP-CLDTOT_TAU1.3_9.4_MISR" not in f
    and "MISRCOSP-CLDTOT_TAU1.3_MISR" not in f
    and "HadISST" not in f
    # https://github.com/E3SM-Project/e3sm_diags/issues/787
    and "CRU-TREFHT" not in f
    and "ERA5-TREFHT" not in f
    and "GPCP_v2.3" not in f
    and "MERRA2-TREFHT" not in f
    and "MERRA2-TREFMNAV" not in f
    and "MERRA2-TREFMXAV" not in f
    and "ERA5-OMEGA" not in f
    and "MERRA2-OMEGA" not in f
    and "MERRA2-U" not in f
    # https://github.com/E3SM-Project/e3sm_diags/issues/852
    and "AOD_550" not in f
]

In [42]:
not_equal_errors

['/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/25-01-10-branch-907-v2-data/zonal_mean_2d_stratosphere/ERA5/ERA5-Q-ANN-global_ref.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/25-01-10-branch-907-v2-data/zonal_mean_2d_stratosphere/ERA5/ERA5-Q-ANN-global_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/25-01-10-branch-907-v2-data/zonal_mean_2d_stratosphere/ERA5/ERA5-Q-DJF-global_ref.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/25-01-10-branch-907-v2-data/zonal_mean_2d_stratosphere/ERA5/ERA5-Q-DJF-global_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/25-01-10-branch-907-v2-data/zonal_mean_2d_stratosphere/ERA5/ERA5-Q-JJA-global_ref.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/25-01-10-branch-907-v2-data/zonal_mean_2d_stratosphere/ERA5/ERA5-Q-JJA-global_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/25-01-10-branch-907-v2-data/zonal_mean_2d_stratosphere/ERA5/ERA5-Q-MAM-global_ref.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-f

Explanation of difference above:

- For the `"Q"` variable,the unit conversion changed from g/kg to g/kg by volume via [4cda0a8](https://github.com/E3SM-Project/e3sm_diags/commit/4cda0a807e496a97392c14d92429bd614c536e4d), and I didn't re-run zonal_mean_2d_stratosphere complete run to update /main results.
- This is good to go.


Results:

We are good to go here. The differences are expected and explained in the comments below.

```
                stat_name  value       pct
0    matching_files_count   1093  0.875801
1      missing_vars_count      0  0.000000
2   mismatch_errors_count     10  0.008013
3  not_equal_errors_count    143  0.114583
4        key_errors_count      0  0.000000
5     missing_files_count      2  0.001603
```

- 1093/1246 matching files
- 10/1246 mismatch errors due to ccb regional subsetting differences
- 143/1246 not equal files -- number of different elements are really small and stats (min, max, mean, sum) are similar, most likely related to the `convert_units()` change to replace `genutil.udunits` with `cf_units.Unit`.
  - Except `Q` variable, differences are explained by not updating the `Q` results for main branch with unit conversion changes
- 2/1246 are missing because `AOD_550` retired.
